In [45]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import time

match_ratio = 0.7
mean_ratio = 0.9


img1 = cv2.imread('voxel-flow-master\\data\\1\\frame_100_1.png', 0)
img2 = cv2.imread('voxel-flow-master\\data\\1\\frame_100_3.png', 0)

print(img1)

sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
kp1a = []
kp2a = []
for i,(m,n) in enumerate(matches):
    if m.distance < match_ratio*n.distance:
        kp1a.append([kp1[m.queryIdx].pt[0], kp1[m.queryIdx].pt[1]])
        kp2a.append([kp2[m.trainIdx].pt[0], kp2[m.trainIdx].pt[1]])

kpX = np.asarray(kp1a)
kpY = np.asarray(kp2a)
dxy = kpX - kpY
dist = np.sqrt(np.sum((kpX-kpY)**2, 1))
#plt.hist(dist, bins=50)
#plt.show()

numSegments = len(dist * 2)
skimg1 = img_as_float(img1)
segments1 = slic(skimg1, numSegments, sigma = 5)
skimg2 = img_as_float(img2)
segments2 = slic(skimg2, numSegments, sigma = 5)




print(dxy)
mean = np.mean(dist)
print(mean)
dist = np.sort(np.absolute(dist-mean))
threshold = dist[int(len(dist)*mean_ratio)]
new_mean = np.mean([x for x in dist if np.absolute(x-mean)<threshold])
print(new_mean)
print(dist)

#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

#plt.imshow(img3)
#plt.show()

[[25 27 32 ...,  0  0  0]
 [25 27 32 ...,  0  0  0]
 [25 27 32 ...,  0  0  0]
 ..., 
 [33 31 29 ...,  2  1  0]
 [17 14 10 ...,  2  1  2]
 [ 8  8  8 ...,  4  4  7]]
[[ -5.45024872e-04   7.62939453e-04]
 [ -1.65243149e-02   7.14111328e-03]
 [  0.00000000e+00   0.00000000e+00]
 [ -1.49917603e-03   4.42504883e-04]
 [ -2.27165222e-03  -5.43212891e-03]
 [ -6.03847504e-02  -4.49066162e-02]
 [  4.74519730e-02  -6.59179688e-03]
 [  4.74519730e-02  -6.59179688e-03]
 [ -3.91052246e-01   4.19113159e-01]
 [  2.63977051e-03  -7.91931152e-03]
 [ -2.13623047e-04   2.13623047e-04]
 [ -2.13623047e-04   2.13623047e-04]
 [ -5.64937592e-02  -7.43103027e-02]
 [  0.00000000e+00   0.00000000e+00]
 [  5.28717041e-03  -3.61633301e-03]
 [  0.00000000e+00   0.00000000e+00]
 [  5.77125549e-02  -1.24893188e-02]
 [  9.84191895e-04   6.86645508e-04]
 [  0.00000000e+00   0.00000000e+00]
 [  2.27485657e-01  -3.63357544e-01]
 [ -8.16345215e-04  -9.62829590e-03]
 [ -7.41577148e-03   1.13159180e-01]
 [ -6.71081543e-02  -7

In [5]:

import numpy as np

x = np.array([[1, 1], [2, 2], [3, 3]])
y = np.array([[4, 4], [6, 6], [8, 8]])
print(x)
print(y)
print(x-y)
print(np.sqrt(np.sum((x-y)**2, 1)))

[[1 1]
 [2 2]
 [3 3]]
[[4 4]
 [6 6]
 [8 8]]
[[-3 -3]
 [-4 -4]
 [-5 -5]]
[ 4.24264069  5.65685425  7.07106781]
